# Этапы проекта

1. [Открытие и исследование файла данных](#first_id)
2. [Предобработка данных](#second)
    * [2.1. Обработка пропусков](#task2_1)
    * [2.2. Замена типа данных](#task2_2)
    * [2.3. Обработка дубликатов](#task2_3)
    * [2.4. Лемматизация](#task2_1)
    * [2.5. Категоризация данных](#task2_4)
3. [Изучение гипотез](#third)
    * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#question_1)
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#question_2)
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#question_3)
    * [Как разные цели кредита влияют на его возврат в срок?](#question_4)
4. [Общий вывод](#fourth)
5. [Чек-лист готовности проекта](#fifth)


## <a id="first_id"></a> Открытие и исследование файла данных  

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

Запишем локальный путь к данным в переменную path.

In [2]:
path = '/Users/segagoose/Documents/YandexPracticum/Предобработка данных'

Прочитаем полученные данные и запишем в переменную data.

In [3]:
try:
    data = pd.read_csv(path + '/data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

Выведем информацию о таблице.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Изучим информацию об уникальных значениях в категориальных столбцах.

In [5]:
for column in [
    'children', 'dob_years', 'education', 'education_id', 'family_status', 
    'family_status_id', 'gender', 'income_type', 'debt'
]:
    print(f'{column}:\n', data[column].value_counts())
    print()

children:
  0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

dob_years:
 35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

education:
 среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное        

Изучим описание данных с помощью метода describe().

In [6]:
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
children,21525.0,NaN,NaN,NaN,0.538908,1.381587,-1.0,0.0,0.0,1.0,20.0
days_employed,19351.0,NaN,NaN,NaN,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,401755.400475
dob_years,21525.0,NaN,NaN,NaN,43.29338,12.574584,0.0,33.0,42.0,53.0,75.0
education,21525,15,среднее,13750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education_id,21525.0,NaN,NaN,NaN,0.817236,0.548138,0.0,1.0,1.0,1.0,4.0
family_status,21525,5,женат / замужем,12380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
family_status_id,21525.0,NaN,NaN,NaN,0.972544,1.420324,0.0,0.0,0.0,1.0,4.0
gender,21525,3,F,14236,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income_type,21525,8,сотрудник,11119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
debt,21525.0,NaN,NaN,NaN,0.080883,0.272661,0.0,0.0,0.0,0.0,1.0


**Вывод**

Мы получили данные хранящиеся в таблице состоящей из 12 колонок и 21525 строк. В таблице содержится информация о заемщиках, в двух колонках days_employed и total_income есть пропуски данных. В колонках children, days_employed и education данные отраженны некорректно, требуется предобработка. В столбцах dob_years и gender также есть артефакты вроде нулевого возраста и пола XNA, но мы эти данные трогать не будем так, как они не влияют на результат нашего исследования.

## Предобработка данных <a id="second"></a> 

### Обработка пропусков <a id="task2_1"></a>

Изучим пропуски данных в столбце 'days_employed'.

In [7]:
data[data['days_employed'].isnull() == 1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пропуски в данных дублируются в столбцах days_employed и total_income по одним и тем же строкам, это говорит о том, что скорее всего при переносе данных по этим людям произошел сбой и данные по работе не подтянулись.

Убираем отрицательные значения из столбцов days_employed и children с помощью модуля числа.

In [8]:
for column in ['days_employed', 'children']:
    data[column] = data[column].apply(abs)

Количество детей "20" принятое за искажение данных меняем на значение 'unknown'.

In [9]:
data.loc[data['children'] == 20, 'children'] = 'unknown'

Пишем цикл для замены пустых значений в двух столбцах и заменяем пропущенные данные медианными значениями

In [10]:
for column in ['days_employed', 'total_income']:
    data = data.sort_values(by=column)

    median_of_days_employed = data[column].median()
    mean_of_days_employed = data[column].mean()

    print(f'Медианное значение колонки {column}:', median_of_days_employed)
    print(f'Среднее значение колонки {column}:', mean_of_days_employed)
    print()
    
    data[column] = data[column].fillna(median_of_days_employed)
    data.info()

Медианное значение колонки days_employed: 2194.220566878695
Среднее значение колонки days_employed: 66914.72890682252

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 17437 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  object 
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.1+ MB
Медианное значение колонки total_income: 145017.

Сортируем таблицу по индексу.

In [11]:
data = data.sort_index()

**Вывод**

Выявленные пропуски данных в столбцах days_employed и total_income были заменены на медианные значения данных колонок. Удаление строк с пропущенными данными могло исказить результаты нашего исследования, также как и среднее значение столбца days_employed существенно завышало показатели.
<br>
В колонке children было отрицательное значение количества детей, его мы заменили на значение со знаком "+" применив модуль числа. Еще один выброс в столбце children - это значение 20, судя по выборке, градация количества детей идет от 0 до 5, значит 20 детей - это ошибка, скорее всего опечатка или некорректный перенос данных, поэтому было принято решение заменить данный параметр на значение 'unknown'.

### Замена типа данных <a id="task2_2"></a>

Создадим цикл для замены типа данных в двух столбцах days_employed и total_income.

In [12]:
for column in ['days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income']:
    data[column] = data[column].astype(np.int32)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  object
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int32 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int32 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int32 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int32 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(6), object(6)
memory usage: 1.6+ MB


**Вывод**

Для удобства и корректности анализа данных, было необходимо перевести вещественный тип данных в целочисленный. Мы имели два столца с типом данных float64: days_employed, total_income и четыре столбца с типом int64. Для сокращения объему занимаемой памяти на 50% мы сконвертировали тип из float64 и int64 в int32, с помощью метода astype(), которым мы и воспользовались.
Метод astype сокращает код и, при его использовании, нет наобходимости проходить циклом по каждому элементу колонки.

### Обработка дубликатов <a id="task2_3"></a>

Приведем дубликаты в столбце education к единому регистру.

In [13]:
data['education'] = data['education'].str.lower()

Проверим результат.

In [14]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Дубликаты можно выявить двумя методами duplicated() и value_counts(), duplicated() отражает только полное совпадение значение и возвращает булевый ответ, тогда как value_counts() покажет нам все уникальные значения в стоблцах, а также количество повторов этих значений.
С помощью метода value_counts() мы выявили, что в столбце education есть множество одних и тех же значений написанных разным регистром, что может повлиять результаты анализа в будущем. Отсутсвие строгих правил заполнения анкет заемщика, могли стать причиной появления данных дубликатов. Для устранения этой проблемы был применен метод str.lower(). Теперь у нас единная градация уровней образования. По столбцам 'family_status' и 'income_type' таких проблем не выявлено.
Дубликаты в нашей таблице не удаляются, так как они являются частью уникальных строк.

### Лемматизация <a id="task2_4"></a>

Cоздаем функцию которая получает фразу и возвращает ее стандартизорованный вариант на основе ключевых слов

In [15]:
m = Mystem()

In [16]:
def lemma_credit_words(phrase):
    if 'автомобиль' in m.lemmatize(phrase):
        return 'приобретение автомобиля'
    if 'жилье' in m.lemmatize(phrase):
        return 'операции с недвижимостью'
    if 'недвижимость' in m.lemmatize(phrase):
        return 'операции с недвижимостью'
    if 'образование' in m.lemmatize(phrase):
        return 'получение образования'
    if 'свадьба' in m.lemmatize(phrase):
        return 'свадьба'
    else:
        return 'другое'

Лемматизируем и стандартизируем данные в столбце `purpose`

In [17]:
data['purpose'] = data['purpose'].apply(lemma_credit_words)

Проверяем перезаписанные данные в столбце `purpose`

In [18]:
data['purpose'].value_counts()

операции с недвижимостью    10840
приобретение автомобиля      4315
получение образования        4022
свадьба                      2348
Name: purpose, dtype: int64

**Вывод**

Изучив уникальные значения в столбце purpose, мы выявили, что цели кредита используют ключевые слова `"автомобиль"`, `"свадьба"`, `"жилье"`, `"недвижимость"` и `"образование"`. Исходя из этого было принято решение привести все значения столбца к стандартным "целям". Для этого мы применили лемматизацию данных, сравнили наличие ключевых слов в лемматизированных фразах и исходя из соответствий, привели фразы к стандартным шаблонам. Итого получлось четыре цели для получения кредита: операции с недвижимостью, приобретение автомобиля, получение образования и свадьба.

### Категоризация данных <a id="task2_5"></a>

Cоздаем переменные с процентилем в 40% и 75%.

In [19]:
quantile_40 = data['total_income'].quantile(0.4)
quantile_75 = data['total_income'].quantile(0.75)

Пишем функцию, которая категоризирует данные по трем уровням дохода.

In [20]:
def income_level(income):
    global quantile_40
    global quantile_75
    if income <= quantile_40:
        return 'низкий доход'
    if income <= quantile_75:
        return 'средний доход'
    else:
        return 'высокий доход'

Создаем новый столбец `income_level` с категориями уровня дохода.

In [21]:
data['income_level'] = data['total_income'].apply(income_level)

Изучаем как распределилась наша выборка по уровням дохода.

In [22]:
data['income_level'].value_counts()

низкий доход     8610
средний доход    7534
высокий доход    5381
Name: income_level, dtype: int64

Пишем функцию для категоризации заемщиков по количеству детей в семье

In [23]:
def family_size(number_of_children):
    if number_of_children == 0:
        return 'семья без детей'
    if number_of_children == 1:
        return 'семья с одним ребенком'
    if number_of_children == 2:
        return 'семья с двумя детьми'
    else:
        return 'многодетная семья'

Создаем новый столбец в нашем датасете с котегориями заемщиков по количеству детей

In [24]:
data['family_size'] = data['children'].apply(family_size)

**Вывод**

Изучив таблицу, определили, что необходимо категоризировать данные в столбце `total_income`. В связи с тем, что в описании данных не указана ни валюта дохода, ни страна, мы не можем с точностью сказать какой порог дохода считается низким, а какой высоким. Исходя из этого принято решение воспользоваться процентилем и разбить данные по выборке на доходы до процентиля 40% - "низкий доход", от 40% до 75% - "средний доход" и свыше 75% - "высокий доход".

## Изучение гипотез <a id="third"></a> 

- Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id="question_1"></a> 

In [25]:
data_pivot = data.pivot_table(index='family_size', values='debt', aggfunc=['sum', 'count'])
data_pivot['ratio'] = data_pivot['sum'] / data_pivot['count']
data_pivot

,sum,count,ratio
,debt,debt,
family_size,,,
многодетная семья,39,456,0.085526
семья без детей,1063,14149,0.075129
семья с двумя детьми,194,2055,0.094404
семья с одним ребенком,445,4865,0.091470


**Вывод**

Коэфициенты должников с различным количеством детей в среднем различаются на 0.01-0.02 пункта, что является некритичным отклонением. Нет ни одного показателя, который бы явно выделялся среди остальных. Зависимость возврата кредита от наличия детей отсутствует.

- Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id="question_2"></a> 

In [26]:
data_pivot = data.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count'])
data_pivot['ratio'] = data_pivot['sum'] / data_pivot['count']
data_pivot

,sum,count,ratio
,debt,debt,
family_status,,,
Не женат / не замужем,274,2813,0.097405
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок отсутствует.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id="question_3"></a> 

In [27]:
data_pivot = data.pivot_table(index='income_level', values='debt', aggfunc=['sum', 'count'])
data_pivot['ratio'] = data_pivot['sum'] / data_pivot['count']
data_pivot

,sum,count,ratio
,debt,debt,
income_level,,,
высокий доход,386,5381,0.071734
низкий доход,702,8610,0.081533
средний доход,653,7534,0.086674


**Вывод**

Зависимость отсутствует.

- Как разные цели кредита влияют на его возврат в срок? <a id="question_4"></a> 

In [28]:
data_pivot = data.pivot_table(index='purpose', values='debt', aggfunc=['sum', 'count'])
data_pivot['ratio'] = data_pivot['sum'] / data_pivot['count']
data_pivot

,sum,count,ratio
,debt,debt,
purpose,,,
операции с недвижимостью,782,10840,0.072140
получение образования,370,4022,0.091994
приобретение автомобиля,403,4315,0.093395
свадьба,186,2348,0.079216


**Вывод**

Разные цели кредита не влияют на возврат кредита в срок.

## Общий вывод <a id="fourth"></a> 

В данном проекте мы проанализировали данные из банка по статистике платежеспособности их клиентов.
<br>
Информация была собрана в 21525 строках и 12 колоноках: 
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита
<br>
<br>
Для того чтобы проверить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, нам потребовалось обработать датасет, для избавления артефактов, выбросов и дубликатов в данных следуюшими способами:
<br>
<br>
* Обработка пропусков
* Замена типа данных
* Обработка дубликатов
* Лемматизация
* Категоризация данных
<br>
<br>
Наша задача была проаналировать датасет и ответить на вопросы:
<br>
<br>
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?
<br>
<br>
Ни одна из гипотез не подтвердилась, соотшения должников к обычным замещикам в среднем различаются на 0.01-0.02 пункта, что является некритичным отклонением. На возврат кредита в срок существенно не влияют наличие детей и их количество, семейное положение, уровень дохода, а также цели кредита.
<br>
<br>
Рекомендуется изменить способ сбора информации о замщиках, а именно:
<br>
<br>
* Стандартизировать заполнение данных по образованию заемщика.
* Стандартизировать заполнение целей кредита, дать возможность выбора цели из списка.
* Выявить как возникают артефакты с отрицательными данными и постараться устранить их.